# LMNA Lipodystrophy, familial partial, type 2 	151660 
Data taken from  [Lanktree M, et al. (2007) Novel LMNA mutations seen in patients with familial partial lipodystrophy subtype 2 (FPLD2; MIM 151660). Clin Genet ;71(2):183-6. PMID:17250669](https://pubmed.ncbi.nlm.nih.gov/17250669/).

The authors write:
The clinical and biochemical attributes of thethree patients are shown in Table 1. Each hadtypical loss of subcutaneous adipose in the limbsand gluteal region, although these findings wereless pronounced for patient C, as often is the case with men. Each had insulin resistance and type 2 diabetes mellitus although none had acanthosis nigricans. Each had high triglycerides and depressed high-density lipoprotein cholesterol. No findings of other laminopathies were present in any patient.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.15


In [2]:
PMID = "PMID:18256394"
title = "Phenotype and course of Hutchinson-Gilford progeria syndrome"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_csv("input/lanktree.tsv", sep='\t')
df.head()

,Patient,Age,Variant,Onset,Sex,BMI,Waist,Facial fat,lipodystrophy.arm,lipodystrophy.gluteal,...,Diabetes.onset,HTN,NM_170707.4,Acanthosis nigricans,cholesterol.total,HDL,LDL,triglycerides,blood.glucose,HBA1Cg
0,A,46,p.D230N,Youth,F,29.3,86,excess,loss,loss,...,44,No,c.688G>A,No,5.13,0.93,2.75,3.20,9.8,7.2
1,B,50,p.R399C,Puberty,F,35.0,80,excess,loss,loss,...,48,No,c.1195C>T,No,4.50,1.12,2.32,2.35,6.5,7.0
2,C,75,p.S573L,Adulthood,M,35.6,100,excess,loss,loss,...,57,Yes(71),c.1718C>T,No,6.04,1.14,4.10,1.75,22.7,9.5


In [4]:
#res = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr)
#print(res)

In [5]:
column_mapper_d = {}
# Each value is above normal limit
bmi_d = {'29.3': 'Obesity',
 '35.0': 'Obesity',
 '35.6': 'Obesity'}
bmiMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=bmi_d)
bmiMapper.preview_column(df['BMI'])
column_mapper_d['BMI'] = bmiMapper

In [6]:
# Facial fat = Increased facial adipose tissue HP:0000287
facialFatMapper = ConstantColumnMapper(hpo_id="HP:0000287", hpo_label="Increased facial adipose tissue")
# facialFatMapper.preview_column(df["Facial fat"])
column_mapper_d['Facial fat'] = facialFatMapper

In [7]:
# Loss of subcutaneous adipose tissue in limbs HP:0003635
subqArmsMapper = ConstantColumnMapper(hpo_id="HP:0003635", hpo_label="Loss of subcutaneous adipose tissue in limbs")
subqArmsMapper.preview_column(df["lipodystrophy.arm"])
column_mapper_d["lipodystrophy.arm"] = subqArmsMapper

In [8]:
# Lipodystrophy HP:0009125
# Currently no more detail available
ldMapper = ConstantColumnMapper(hpo_id="HP:0009125", hpo_label="Lipodystrophy")
ldMapper.preview_column(df["lipodystrophy.arm"])
column_mapper_d['lipodystrophy.arm'] = ldMapper

In [9]:
# Acanthosis nigricans HP:0000956
anMapper = ConstantColumnMapper(hpo_id="HP:0009125", hpo_label="Lipodystrophy", excluded=True)
anMapper.preview_column(df["Acanthosis nigricans"])
column_mapper_d['Acanthosis nigricans'] = anMapper

In [10]:
# Type II diabetes mellitus  HP:0005978
dmMapper = ConstantColumnMapper(hpo_id="HP:0005978", hpo_label="Type II diabetes mellitus")
dmMapper.preview_column(df["Diabetes.onset"])
column_mapper_d["Diabetes.onset"] = dmMapper

In [11]:
# Hypertriglyceridemia HP:0002155
htMapper = ConstantColumnMapper(hpo_id="HP:0002155", hpo_label="Hypertriglyceridemia")
htMapper.preview_column(df["triglycerides"])
column_mapper_d["triglycerides"] = htMapper

In [12]:
# Hypercholesterolemia HP:0003124
cholMapper = ConstantColumnMapper(hpo_id="HP:0003124", hpo_label="Hypercholesterolemia")
cholMapper.preview_column(df['cholesterol.total'])
column_mapper_d["cholesterol.total"] = cholMapper

In [13]:
# Decreased HDL cholesterol concentration HP:0003233
hdlMapper = ConstantColumnMapper(hpo_id="HP:0003233", hpo_label="Decreased HDL cholesterol concentration")
hdlMapper.preview_column(df["HDL"])
column_mapper_d["HDL"] = hdlMapper

In [14]:
# Hyperglycemia HP:0003074 -- assume upper limit of normal of 5.6 mmol/L
glucoseMapper = ConstantColumnMapper(hpo_id="HP:0003074", hpo_label="Hyperglycemia")
glucoseMapper.preview_column(df['blood.glucose'])
column_mapper_d["blood.glucose"] = glucoseMapper

In [15]:
# Elevated hemoglobin A1c HP:0040217 -- assume normal range at 4.5-5.7%
hba1cg_d = {'7.2': 'Elevated hemoglobin A1c',
 '7.0': 'Elevated hemoglobin A1c',
 '9.5': 'Elevated hemoglobin A1c'}
hba1cgMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=hba1cg_d)
hba1cgMapper.preview_column(df['HBA1Cg'])
column_mapper_d['HBA1Cg'] = hba1cgMapper

In [16]:
ageMapper = AgeColumnMapper.by_year(column_name="Age")
#ageMapper.preview_column(df["Age"])
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")
#sexMapper.preview_column(df["Sex"])

In [17]:
LMNA_transcript = "NM_170707.4"
vvalidator = VariantValidator(genome_build="hg38", transcript=LMNA_transcript)
var_d = {}
for v in df["NM_170707.4"].unique():
    print(f'"{v}"')
    var = vvalidator.encode_hgvs(v)
    var.set_heterozygous()
    print(var)
    var_d[v] = var

"c.688G>A"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_170707.4%3Ac.688G>A/NM_170707.4?content-type=application%2Fjson
chr1:156134853G>A
"c.1195C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_170707.4%3Ac.1195C>T/NM_170707.4?content-type=application%2Fjson
chr1:156136251C>T
"c.1718C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_170707.4%3Ac.1718C>T/NM_170707.4?content-type=application%2Fjson


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_d, 
                        individual_column_name="Patient", 
                        agemapper=ageMapper, 
                        va
                        sexmapper=sexMapper,
                        metadata=metadata)
omim_id = "OMIM:151660"
omim_label = "Lipodystrophy, familial partial, type 2"
 	 lipodystrophy = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=lipodystrophy)

## Excluded phenotypes
The authors write that No findings of other laminopathies were present in any patient.
Therefore, we exclude several characteristic findings

- Proptosis (HP:0000520); 
- Failure to thrive (HP:0001508); 
- Micrognathia (HP:0000347); 
- Alopecia (HP:0001596); 
- Short stature (HP:0004322); 
- Scleroderma (HP:0100324); 
- Prominent superficial blood vessels (HP:0007394); 
- Mottled pigmentation (HP:0001070)
- Progeroid facial appearance HP:0005328

In [18]:
for i in individuals:
    proptosis = HpTerm(hpo_id="HP:0000520", label="Proptosis", excluded=True)
    i.add_hpo_term(proptosis)
    micrognathia = HpTerm(hpo_id="HP:0000347", label="Micrognathia", excluded=True)
    i.add_hpo_term(micrognathia)
    sqfat = HpTerm(hpo_id="HP:0003758", label="Reduced subcutaneous adipose tissue", excluded=True)
    i.add_hpo_term(sqfat)
    alopecia = HpTerm(hpo_id="HP:0001596", label="Alopecia", excluded=True)
    i.add_hpo_term(alopecia)
    vessels = HpTerm(hpo_id="HP:0007394", label="Prominent superficial blood vessels", excluded=True)
    i.add_hpo_term(vessels)
    ftt = HpTerm(hpo_id="HP:0001508", label="Failure to thrive", excluded=True)
    i.add_hpo_term(ftt)
    sst = HpTerm(hpo_id="HP:0004322", label="Short stature", excluded=True)
    i.add_hpo_term(sst)
    bmt = HpTerm(hpo_id="HP:0004349", label="Reduced bone mineral density", excluded=True)
    i.add_hpo_term(bmt)
    prog = HpTerm(hpo_id="HP:0005328", label="Progeroid facial appearance", excluded=True)
    i.add_hpo_term(bmt)

NameError: name 'individuals' is not defined